<a href="https://colab.research.google.com/github/TranPhu1999/Wordle_solver/blob/main/wordle_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference:
- 3blue1brown video: https://www.youtube.com/watch?v=v68zYyaEmEA&t=848s
- 3blue1brown repo: https://github.com/3b1b/videos/tree/870a6cbf30938793f93a2c9235c82bdeed31c7c6/_2022/wordle

# Install

In [1]:
!pip install -r /content/drive/MyDrive/Projects/Wordle_solver/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for english_words: filename=english_words-2.0.1-py3-none-any.whl size=8196240 sha256=c1f65c91d3a9438109a69ddebe4c78ce017a1a88350511c4f891249730200392
  Stored in directory: /root/.cache/pip/wheels/f0/e6/d9/16a123647999fe535f03a36e7af23eef203736d84c7ca25b0b
Successfully built english_words


# Dev_v1

In [25]:
# import
import requests
from tqdm import tqdm as ProgressDisplay
from itertools import product
from math import log
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from english_words import get_english_words_set
import numpy as np

# Disable InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [53]:
# Global variable
web2lowerset = get_english_words_set(['web2'], lower=True, alpha =False) # gcide / web2
web2lowerset = list(web2lowerset)
common_letter_rank = "etaonrishdlfcmugypwbvkjxzq"


CORRECT, PRESENT, ABSENT = np.uint8(2), np.uint8(1), np.uint8(0)

In [27]:
def check_answer(guess_word: str, seed: int, text_size: int) -> list:
  # Check answer from API and return next pattern
  query = f"https://wordle.votee.dev:8000/random?guess={guess_word}&seed={seed}&size={text_size}"
  r = requests.get(query, verify = False)
  result = [check["result"] for check in r.json()]
  return result

In [28]:
# check_answer("thefo",1234,5)

In [55]:
def get_the_remain_words_list(guess_word: str, pattern: list, allowed_words_list: list) -> list:
  remain_list = []

  for word in allowed_words_list:
    check = True
    for i, possible in enumerate(pattern): # "absent": 0, "present": 1, "correct": 2,
      if (possible == ABSENT and guess_word[i] in word) or \
      (possible == PRESENT and guess_word[i] not in word) or \
      (possible == CORRECT and guess_word[i] != word[i]):
        check = False
    if check:
      remain_list.append(word)
  return remain_list

In [56]:
# get_the_remain_words_list("theao",[2,2,2,0,0],[x for x in web2lowerset  if len(x) == 5])

In [57]:
def calculate_entropy(word, all_pattern: list, allowed_words_list: list)-> float:
  # Return entropy of the correspond word, check the details explanation here: https://youtu.be/v68zYyaEmEA
  # For a particular word, check for all possible pattern and their probability
  # Then calculate the entropy for each of these word with formular of information theory E = sum(px*log(1/px))

  entropy = 0
  for pattern in all_pattern:
    word_basket = get_the_remain_words_list(word, pattern, allowed_words_list)
    if len(word_basket) > 0:
      px = len(word_basket)/len(allowed_words_list)
      entropy += px * log(1/px,2)
  return entropy

In [58]:
def get_best_guess(allowed_words_list: list, all_pattern: list)-> list:
  # from the allow word list, get the word that give the highest entropy
  # higher entropy mean it wil help split the possibility word even smaller
  highest_entropy = 0
  result_index = 0
  for i, word in enumerate(ProgressDisplay(allowed_words_list)):
    entropy = calculate_entropy(word, all_pattern, allowed_words_list)
    if entropy > highest_entropy:
      highest_entropy = entropy
      result_index = i

  return result_index

In [59]:
def recursive_guess(inital_guess: str, allowed_words_list: list, all_pattern: list, seed: int) -> str:
  # recursive function, shrink the allowed_words_list down untill there is one word left or the guess was correct
  if len(allowed_words_list) == 1:
    return allowed_words_list[0]

  guess_pattern = check_answer(inital_guess, seed, len(inital_guess))
  print(f"\ninitial guess: {inital_guess}")
  print(f"guess_pattern: {guess_pattern}")
  if set(guess_pattern) == set(["correct"]):
    return inital_guess
  else:
    #if not true, filter the allowed_words_list with the guess_pattern above
    pattern = {"absent": ABSENT, "present": PRESENT, "correct": CORRECT}
    guess_pattern = [pattern[x] for x in guess_pattern]
    remain_words = get_the_remain_words_list(inital_guess, guess_pattern, allowed_words_list)
    best_guess_index = get_best_guess(remain_words, all_pattern)
    best_guess = remain_words.pop(best_guess_index)
    return recursive_guess(best_guess, remain_words, all_pattern, seed)

In [60]:
def wordle_solver(seed: int, text_size: int) -> str:
  all_pattern = np.array(list(product(*[[ABSENT ,PRESENT ,CORRECT]]*text_size)),dtype=int) # "absent": 0, "present": 1, "correct": 2,
  # main function solve wordle

  allowed_text = []

  for text in web2lowerset:
    if len(text) == text_size and text not in allowed_text:
      allowed_text.append(text)

  answer = recursive_guess(inital_guess = common_letter_rank[:text_size],\
                           allowed_words_list = allowed_text, all_pattern=all_pattern, seed = seed)

  return answer

In [ ]:
%%time
wordle_solver(1234, 5)


initial guess: etaon
guess_pattern: ['present', 'present', 'absent', 'absent', 'absent']


 65%|██████▌   | 264/405 [01:03<00:30,  4.69it/s]